In [1]:
# Import necessary libraries
from pymatgen.ext.matproj import MPRester
from pymatgen.io.ase import AseAtomsAdaptor
from ase import Atoms
from ase.build import bulk, surface, make_supercell
from ase.io import write
from ase.visualize import view
from scipy.spatial.transform import Rotation as R
import parmed as pmd
from openmm import openmm, unit
from openmm import app
import nglview as nv
import numpy as np
import os
from os import mkdir, makedirs
from ase.calculators.lammpsrun import LAMMPS
from ase.io.lammpsdata import write_lammps_data
from MDAnalysis import Universe
import utils


In [33]:
import os
from ase.io import read, write
from ase.build import make_supercell, surface
from ase.build.tools import sort
from ase.visualize import view
import numpy as np

# Function to write LAMMPS data file with atom charges
def write_lammps_data(filename, atoms, atom_charges):
    # Sort atoms by type (Ni first, then O)
    sorted_atoms = sorted(zip(atoms.get_positions(), atoms.get_chemical_symbols(), atom_charges), key=lambda x: (x[1], x[0].tolist()))

    num_atoms = len(atoms)
    num_atom_types = len(set(atoms.get_chemical_symbols()))

    cell = atoms.get_cell()
    positions, symbols, charges = zip(*sorted_atoms)

    with open(filename, 'w') as f:
        f.write("LAMMPS data file\n\n")
        f.write(f"{num_atoms} atoms\n")
        f.write(f"{num_atom_types} atom types\n\n")
        f.write(f"0.0 {cell[0,0]} xlo xhi\n")
        f.write(f"0.0 {cell[1,1]} ylo yhi\n")
        f.write(f"0.0 {cell[2,2]} zlo zhi\n\n")
        f.write("Masses\n\n")
        
        mass_dict = {'Ni': 58.6934, 'O': 15.999}  # Atomic masses
        type_dict = {'Ni': 1, 'O': 2}
        
        for symbol in ['Ni', 'O']:
            f.write(f"{type_dict[symbol]} {mass_dict[symbol]}\n")
        
        f.write("\nAtoms\n\n")
        
        for i, (pos, symbol, charge) in enumerate(zip(positions, symbols, charges)):
            type_id = type_dict[symbol]
            f.write(f"{i+1} 0 {type_id} {charge} {pos[0]} {pos[1]} {pos[2]}\n")

# Define input and output directories
input_dir = './INPUT'
output_dir = './OUTPUT'

# Ensure the OUTPUT directory exists
os.makedirs(output_dir, exist_ok=True)

# Read the CIF file from the INPUT directory
nio = read(os.path.join(input_dir, 'NiO_mp-19009.cif'))

# Ensure the cell is cubic
a = nio.cell[0, 0]
nio.set_cell([a, a, a])
nio.set_pbc(True)

# Create a (1 1 0) slab of NiO with 8 layers and 0 Å initial vacuum
slab = surface(nio, indices=(1, 1, 0), layers=12, vacuum=0)

# Create a supercell from the slab
supercell = make_supercell(slab, [[10, 0, 0], [0, 14, 0], [0, 0, 1]])

# Sort the supercell structure
supercell_sorted = sort(supercell)

# Add 20 Å of vacuum on one side and translate 3 Å above the initial z position
cell = supercell_sorted.cell
cell[2, 2] += 23
supercell_sorted.set_cell(cell, scale_atoms=False)
supercell_sorted.translate([0, 0, 3])

# Define atom charges (example: Ni = 2.0, O = -2.0)
atom_charges = [2.0 if symbol == 'Ni' else -2.0 for symbol in supercell_sorted.get_chemical_symbols()]

# Write the ASE atoms object to a POSCAR file
write(os.path.join(output_dir, 'POSCAR'), supercell_sorted, format='vasp')

# Write the ASE atoms object to an XYZ file
write(os.path.join(output_dir, 'NiO_supercell.xyz'), supercell_sorted)

# Write the ASE atoms object to a LAMMPS data file with charges
write_lammps_data(os.path.join(output_dir, 'NiO_supercell.lmp'), supercell_sorted, atom_charges)

# Visualize the structure
view(supercell_sorted)

print(f"NiO supercell structure saved as 'POSCAR', 'NiO_supercell.xyz', and 'NiO_supercell.lmp' in the '{output_dir}' directory.")


NiO supercell structure saved as 'POSCAR', 'NiO_supercell.xyz', and 'NiO_supercell.lmp' in the './OUTPUT' directory.


In [6]:
# Load the LigParGen topology and XYZ files for 4-MePAcZ
#This was mostly for tests, probably not relevant now, use PackMol for such tasks now. It is useful for aligning the phosphate groups however, so its worth keeping
ligpargen_itp = './INPUT/INPUT.itp'
ligpargen_xyz = './INPUT/INPUT.xyz'

# Load the topology and coordinates using ParmEd
p_ligand_structure = pmd.load_file(ligpargen_itp)
p_ligand_structure.coordinates = pmd.load_file(ligpargen_xyz).coordinates


# Identify the phosphorus atom in the ligand
phosphorus_atom = next(atom for atom in p_ligand_structure.atoms if atom.element == 15)

# Calculate the center of the NiO slab
slab_center_x = supercell.cell[0, 0] / 2
slab_center_y = supercell.cell[1, 1] / 2

# Translate the entire ligand structure to the center above the NiO surface
translation_vector = [slab_center_x, slab_center_y, max(p_nio_structure.coordinates[:, 2]) + 2.0]  # Adjust z as needed
p_ligand_structure.coordinates += translation_vector

# Calculate the vector from the phosphorus atom to the target point on the NiO surface
target_position = np.array([supercell.cell[0, 0] / 2, supercell.cell[1, 1] / 2, max(p_nio_structure.coordinates[:, 2]) + 3.0])
current_position = p_ligand_structure.coordinates[phosphorus_atom.idx]
align_vector = target_position - current_position

# Determine the current direction of the phosphorus atom
current_vector = np.array([0, 0, 1])  # Assuming the molecule is initially oriented along the z-axis

# Calculate the rotation matrix to align the current vector with the align vector
rotation_axis = np.cross(current_vector, align_vector)
rotation_angle = np.arccos(np.dot(current_vector, align_vector) / (np.linalg.norm(current_vector) * np.linalg.norm(align_vector)))
rotation_matrix = R.from_rotvec(rotation_angle * rotation_axis / np.linalg.norm(rotation_axis)).as_matrix()

# Apply the rotation to the entire Me4PACz molecule
p_ligand_structure.coordinates = np.dot(p_ligand_structure.coordinates - current_position, rotation_matrix) + current_position

# Combine NiO and Me4PACz structures
combined_structure = p_ligand_structure + p_nio_structure 

# Add particles for combined_structure (Me4PACz + NiO)
#for atom in combined_structure.atoms:
#    system.addParticle(atom.mass * unit.amu)

combined_structure.visualize()

NGLWidget()

In [9]:
from ase.io.lammpsdata import write_lammps_data
from ase.build import sort


def parmed_to_ase(parmed_structure):
    # Extract coordinates and elements
    coordinates = parmed_structure.coordinates
    symbols = [atom.element_name for atom in parmed_structure.atoms]
    
    # Create ASE Atoms object
    cell = np.array([
        [supercell.cell[0, 0], 0, 0],
        [0, supercell.cell[1, 1], 0],
        [0, 0, supercell.cell[2, 2] + 10]  # Adding some padding to the z-axis
    ])
    ase_atoms = Atoms(symbols=symbols, positions=coordinates, cell=cell, pbc=True)
    return ase_atoms


ase_atoms_final = parmed_to_ase(combined_structure)

# Write the ASE Atoms object to a LAMMPS data file
#write("./LAMMPS_Output/system.data", ase_atoms_final, format="lammps-data")
write("./LAMMPS_Output/system.xyz", ase_atoms_final)

write_lammps_data("LAMMPS_Output/system.data", ase_atoms_final, masses=True, units='real', atom_style='full' )
utils.lammps_data_convert_to_real_numbers('LAMMPS_Output/system.data')

ase_atoms_final=sort(ase_atoms_final)
write("./LAMMPS_Output/POSCAR", ase_atoms_final)

print("LAMMPS data file written successfully.")


LAMMPS data file written successfully.


In [ ]:
#Outdated Code,saved in case its needed

# Initialize the Materials Project REST API client

with MPRester("M5gsyRYge3DgdJjf4NRxiDpXOvDHE3oB") as mpr:
    # Fetch the structure for NiO with mp-19009
    nio_structure = mpr.get_structure_by_material_id("mp-19009")


# Convert pymatgen structure to ASE Atoms
positions = nio_structure.cart_coords
numbers = [site.specie.number for site in nio_structure]
atoms = Atoms(numbers=numbers, positions=positions, cell=nio_structure.lattice.matrix, pbc=True)

# Convert pymatgen structure to ASE Atoms using AseAtomsAdaptor
ase_atoms = AseAtomsAdaptor.get_atoms(nio_structure)

# Build the NiO rock salt structure using ASE
nio_rocksalt = bulk('NiO', 'rocksalt', a=4.17)

# Create a (1 0 -1) slab of NiO
slab = surface(nio_rocksalt, indices=(1, 0, 0), layers=8, vacuum=15)

# Create a supercell from the slab
supercell = make_supercell(slab, [[6, 0, 0], [0, 6, 0], [0, 0, 1]])

# Write the ASE atoms object to an XYZ file
write("NiO_supercell.xyz", nio_rocksalt, format="xyz")

# Convert XYZ to PDB using Open Babel
!obabel NiO_supercell.xyz -O NiO_supercell.pdb

# Load the PDB file using ParmEd
p_nio_structure = pmd.load_file("NiO_supercell.pdb")

# Load the LigParGen topology and XYZ files for 4-MePAcZ
#ligpargen_itp = 'INPUT.itp'
#ligpargen_xyz = 'INPUT.xyz'

# Load the topology and coordinates using ParmEd
#p_ligand_structure = pmd.load_file(ligpargen_itp)
#p_ligand_structure.coordinates = pmd.load_file(ligpargen_xyz).coordinates